In [1]:
import json
from itertools import chain
from math import ceil

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

/home/ubuntu/anaconda2/envs/fashion/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator()
gen = datagen.flow_from_directory('images',
                                  target_size=(224, 224),
                                  class_mode=None, 
                                  shuffle=False,
                                  batch_size=32)

Found 100000 images belonging to 1 classes.


In [3]:
# num_images = len(gen.filenames)
num_images = 10_000
num_images

10000

In [4]:
%%time
batches = []
for i in range(ceil(num_images / gen.batch_size)):
    batches.append(gen.next())
    
img_arrays = np.array(list(chain(*batches)))
del batches
img_arrays = img_arrays[:num_images]

CPU times: user 48.3 s, sys: 5.97 s, total: 54.3 s
Wall time: 54.3 s


In [5]:
img_arrays.shape

(10000, 224, 224, 3)

In [6]:
img_ids = np.array([int(fn[6:-4]) for fn in gen.filenames[:num_images]])
img_ids.shape

(10000,)

# id-to-labels mapping

In [7]:
with open('data/train.json') as fp:
    train = json.load(fp)
    
df = pd.DataFrame(train['annotations'][:100000])
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df['labelId']),
                          columns=mlb.classes_,
                          index=df.index))
df = df.drop('labelId', axis=1)
df.imageId = df.imageId.astype(int)
df = df.set_index('imageId')
df.head()

,1,10,100,101,102,103,104,105,106,107,...,90,91,92,93,94,95,96,97,98,99
imageId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.shape

(100000, 227)

In [9]:
img_labels = df.loc[img_ids]
img_labels.shape

(10000, 227)

## arrays size in MB

In [10]:
img_arrays.size * img_arrays.itemsize / 1024**2

5742.1875

# train-test split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(img_arrays, img_labels)

In [13]:
for a in X_train, X_test, y_train, y_test:
    print(a.shape)

(7500, 224, 224, 3)
(2500, 224, 224, 3)
(7500, 227)
(2500, 227)


# "test test" images

In [2]:
gen_test = ImageDataGenerator().flow_from_directory('images_test',
                                                    target_size=(224, 224),
                                                    class_mode=None, 
                                                    shuffle=False,
                                                    batch_size=32)
num_images_test = len(gen_test.filenames)

Found 39706 images belonging to 1 classes.


In [18]:
# %%time
# def make_arrays(gen, n_images):
#     batches = []
#     for i in range(ceil(n_images / gen.batch_size)):
#         batches.append(gen.next())

#     img_arrays = np.array(list(chain(*batches)))
#     del batches
#     img_arrays = img_arrays[:num_images]
#     return img_arrays

# img_arrays_test = make_arrays(gen_test, num_images_test)

MemoryError: 

In [ ]:
start_batch = 0
n_images = num_images_test

batches = []
for i in range(ceil